<a href="https://www.kaggle.com/code/minanalminazor/predict-titanic-survival-with-logisticregression?scriptVersionId=140680139" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# Load the datas
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")

train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
# Update sex column to numerical
train_data['Sex'] = train_data['Sex'].replace({'female': 1, 'male': 0})
test_data['Sex'] = test_data['Sex'].replace({'female': 1, 'male': 0})

In [5]:
# Fill the nan values in the age column
train_data['Age'].fillna(value=train_data['Age'].mean(),inplace=True)
test_data['Age'].fillna(value=test_data['Age'].mean(),inplace=True)

In [6]:
# Create a first class column
train_data['FirstClass'] = train_data['Pclass'].apply(lambda x: 1 if x == 1 else 0)
test_data['FirstClass'] = test_data['Pclass'].apply(lambda x: 1 if x == 1 else 0)

# Create a second class column
train_data['SecondClass'] = train_data['Pclass'].apply(lambda x: 1 if x == 2 else 0)
test_data['SecondClass'] = test_data['Pclass'].apply(lambda x: 1 if x == 2 else 0)

In [7]:
# Select the desired features
features = train_data[["Sex", "Age", "FirstClass", "SecondClass"]]
survival = train_data["Survived"]
# Perform train, test, split
X_train,X_test,y_train,y_test = train_test_split(features,survival,test_size=0.20,random_state=43)

In [8]:
scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=features.columns)
X_test = pd.DataFrame(scaler.transform(X_test), columns=features.columns)

In [9]:
# Create and train the model
model = LogisticRegression()
model.fit(X_train,y_train)

LogisticRegression()

In [10]:
# Score the model on the train data
print(model.score(X_train,y_train))

# Score the model on the test data
print(model.score(X_test,y_test))

0.8117977528089888
0.7374301675977654


In [11]:
# Analyze the coefficients
print(list(zip(['Sex','Age','FirstClass','SecondClass'],model.coef_[0])))

[('Sex', 1.373466543810655), ('Age', -0.4413572335520608), ('FirstClass', 1.0908196068972507), ('SecondClass', 0.5748002145259937)]


In [12]:
test_data_pred = test_data[["Sex", "Age", "FirstClass", "SecondClass"]]

In [13]:
# Make survival predictions!
preds = model.predict(test_data_pred)

In [14]:
test_data['Survived'] = preds

In [15]:
(test_data[['PassengerId', 'Survived']]).to_csv("submission.csv",index=False)